# MLGO: Deep Neural Network for Gene Ontology Prediction

## Preliminary Analysis Report

**By: [Tony Kabilan Okeke](mailto:tko35@drexel.edu)**

In [9]:
# Imports
import pandas as pd
import feather as ft
import yaml

# Load config
with open('config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [11]:
# Read data
df = ft.read_dataframe(config['paths']['processed'])

In [12]:
df

,GSE,Prune2,Eps8l1,Hjurp,Ermap,C920006O11Rik,Gm40518,Fgl1,3110053B16Rik,Slfn14,...,Vmn1r186,V49039,V49040,V49041,V49042,V49043,V49044,V49045,V49046,GO_terms
0,GSE100035,-88749.742424,-195.575758,-1229.454545,-72.030303,-94.030303,-1548.545455,-73.833333,-60.439394,-25.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GO:0005622;GO:0043226;GO:0043229;GO:0005737;GO...
1,GSE100037,-43.400000,312.495000,-600.055000,-43.525000,-423.175000,-740.235000,271.465000,-277.195000,-144.180000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GO:0006807;GO:0008152;GO:0009987;GO:0044237;GO...
2,GSE100048,1.000000,1.000000,1.500000,0.966667,0.966667,0.966667,2.000000,1.500000,1.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GO:0004984;GO:0007606;GO:0007608;GO:0004888;GO...
3,GSE100060,1.000000,-337.976190,1393.714286,149.904762,36.142857,181.357143,56.976190,-91.261905,30.404762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GO:0005622;GO:0005737;GO:0006629;GO:0043229;GO...
